In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import numpy as np
import logging
import copy
import random
import collections
import os

from time import gmtime, strftime
from matplotlib import pyplot as plt
from functools import reduce



logging.basicConfig(level=logging.INFO)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Cooperate Value
coop = 'C'

# Defect value
dfct = 'D'

# Von Newmann type
vonNewmann = 'vonNewmann'

# Moore type
moore = 'moore'

vonNewmannSize = 4
mooreSize = 8

class Player:
    """Class representing a player that will take part in the problems"""

    # Definition of elements for a player
    neighbourhood = None
    game = None
    payoff = 0
    strategy = None
    strategyHistory = None

    def __init__(self, neighbourhoodSize, strategy, game):
        #logging.debug('Creating instance of a player')
        self.neighbourhood = np.empty([neighbourhoodSize])
        self.strategy = strategy
        self.game = game
        self.strategyHistory = [self.strategy]
        #logging.debug('Instance of player created successfully')

    def play(self):
        """
        Runs the game with all of the neighbours and computes the payoff for the current iteration.
        """
        payoff = 0
        for i in range(0,len(self.neighbourhood)):
            payoff += self.game.run([self.strategy, self.neighbourhood[i].strategy])[0]
        self.payoff = payoff

    def imitate(self):
        logging.debug('Imitating ...')
        logging.debug('Current Strategy:' + self.strategy + ", own payoff: " + str(self.payoff))
        self.strategy = self.maxPayoffNeighbour().strategy
        self.strategyHistory.append(self.strategy)
        logging.debug('New Strategy:' + self.strategy + ", Neighbour payoff: " + str(self.maxPayoffNeighbour().payoff))
    
    def maxPayoffNeighbour(self):
        mx = self.neighbourhood[0]
        for i in range(1,len(self.neighbourhood)):
            if self.neighbourhood[i].payoff > mx.payoff:
                mx = self.neighbourhood[i]
        return mx

class Game:
    """Common base class for all games"""

    # Definition of elements for a game
    numPlayers = 2  # Number of players. Default 2
    matrix = None  # Game Matrix
    strategies = None  # Possible strategy values for the game. Stored as a dictionary with each entry containing [value, index]. The index corresponds to the one in the matrix of the game

    def __init__(self, numPlayers, matrix, strategies):
        logging.debug('Creating instance of game')
        self.numPlayers = numPlayers
        self.matrix = matrix
        self.strategies = strategies
        logging.debug('Instance of game created')

    def run(self, strategies):
        """Executes the current game. Given the value of the game matrix and strategies chosen returns the value for both players"""

        #logging.debug('Playing a game')
        return self.matrix[self.strategies[strategies[0]],
                           self.strategies[strategies[1]]]


class Simulator:
    """Simulator class in charge of executing the main logic of the application"""

    # Definition of elements for the simulator
    lattice = None
    game = None
    avgValue = None  # Value used in the terminate computation and the required plots
    lastLatticeStrategy = None
    latticeSize = None
    neighbourhoodType = None
    cooperationLevelHistory = None
    cooperationLevelData = []
    reComputeCurrentLatticeStrategy = None
    currentLatticeStrategyList = None
    coopLvlShortHistory = None
    maxLoops = 300
    coopLvlShortSize = None
    minStdDev = 0.02
    directoryOutput = None

    def __init__(self, latticeSize, game, neighbourhoodSize, neighbourhoodType, avgValue):
        self.game = game
        self.avgValue = avgValue
        self.coopLvlShortSize = int(self.avgValue / 3)
        self.maxLoops = self.avgValue * 5
        self.latticeSize = latticeSize
        self.neighbourhoodType = neighbourhoodType
        self.directoryOutput = strftime("%Y-%m-%d_%H-%M-%S", gmtime())
        self.cooperationLevelHistory = collections.deque(maxlen=self.avgValue)
        self.coopLvlShortHistory = collections.deque(maxlen=int(self.coopLvlShortSize))
        self.reComputeCurrentLatticeStrategy = True
        logging.info('Creating instance of simulator')
        self.initLattice(neighbourhoodSize)
        self.computeNeighbourhoods()
        logging.info('Instance of simulator created successfully')

    def initLattice(self, neighbourhoodSize):
        """Initialize the lattice with a set of nxn different players"""
        logging.debug('Initializing lattice for simulator')
        self.lattice = np.empty([self.latticeSize, self.latticeSize], dtype=object)
        for i in range(0, self.latticeSize):
            for j in range(0, self.latticeSize):
                self.lattice[i, j] = Player(neighbourhoodSize,
                        self.randomStrategy(), self.game)
        logging.debug('Players created in lattice for simulator')

    def randomStrategy(self):
        if random.uniform(0, 1) < 0.5:
            return coop
        return dfct

    def computeNeighbourhoods(self):
        """Initialize the neighbourhoods for the players of the simulation"""
        logging.debug('Computing neighbours for players in lattice')
        for i in range(self.latticeSize):
            for j in range(self.latticeSize):
                self.lattice[i, j].neighbourhood = self.computeNeighbours(i, j, len(self.lattice))
        logging.debug('Neighbours successfully assigned for players in lattice')

    def computeNeighbours(self, row, col, size):
        #logging.debug('Computing neighbours for player' + str(row) + ',' + str(col) + ' in lattice')
        neighbours = None
        if self.neighbourhoodType == vonNewmann:
            neighbours = np.empty([4], dtype=object)
            neighbours[0] = copy.copy(self.lattice[row % self.latticeSize][(col - 1) % self.latticeSize])
            neighbours[1] = copy.copy(self.lattice[row % self.latticeSize][(col + 1) % self.latticeSize])
            neighbours[2] = copy.copy(self.lattice[(row - 1) % self.latticeSize][col % self.latticeSize])
            neighbours[3] = copy.copy(self.lattice[(row + 1) % self.latticeSize][col % self.latticeSize])
        if self.neighbourhoodType == moore:
            neighbours = np.empty([8], dtype=object)
            neighbours[0] = copy.copy(self.lattice[(row - 1) % self.latticeSize][(col - 1) % self.latticeSize])
            neighbours[1] = copy.copy(self.lattice[(row - 1) % self.latticeSize][col % self.latticeSize])
            neighbours[2] = copy.copy(self.lattice[(row - 1) % self.latticeSize][(col + 1) % self.latticeSize])
            neighbours[3] = copy.copy(self.lattice[row % self.latticeSize][(col - 1) % self.latticeSize])
            neighbours[4] = copy.copy(self.lattice[row % self.latticeSize][(col + 1) % self.latticeSize])
            neighbours[5] = copy.copy(self.lattice[(row + 1) % self.latticeSize][(col - 1) % self.latticeSize])
            neighbours[6] = copy.copy(self.lattice[(row + 1) % self.latticeSize][col % self.latticeSize])
            neighbours[7] = copy.copy(self.lattice[(row + 1) % self.latticeSize][(col + 1) % self.latticeSize])
        return neighbours
    
    def currentLatticeStrategy(self):
        if self.reComputeCurrentLatticeStrategy:
            self.currentLatticeStrategyList = [p.strategy for p in self.lattice.flat]
        return self.currentLatticeStrategyList

    def terminate(self, loop):
        """Determine whether a stable state has been reached and it's good to stop"""
        return (np.std(self.coopLvlShortHistory) < self.minStdDev and loop > self.avgValue) or loop > self.maxLoops

    def run(self):
        logging.info('Starting to run simulator')
        
        if not os.path.exists(self.directoryOutput):
            os.makedirs(self.directoryOutput)
        
        generateChart(self.currentLatticeStrategy(), True, False, './' + self.directoryOutput + '/initial')
        
        loop = 0
        
        self.lastLatticeStrategy = self.currentLatticeStrategy()
                      
        for i in range(self.latticeSize):
            for j in range(self.latticeSize):
                self.lattice[i][j].play()
        self.computeNeighbourhoods()
        
        while not self.terminate(loop):
            self.lastLatticeStrategy = self.currentLatticeStrategy()
            for i in range(self.latticeSize):
                for j in range(self.latticeSize):
                    self.lattice[i][j].imitate()
                    self.lattice[i][j].play()
            self.computeNeighbourhoods()
            self.reComputeCurrentLatticeStrategy = True
            self.cooperationLevelHistory.append(self.cooperationLevel())
            self.cooperationLevelData.append(reduce(lambda x, y: x + y, self.cooperationLevelHistory) / len(self.cooperationLevelHistory))
            self.coopLvlShortHistory.append(self.cooperationLevel())
            logging.debug('Iteration: '+ str(loop))
            loop = loop + 1
            logging.debug('Std deviation: ' + str(np.std(self.payoffs())))
            logging.debug('Coop Level: ' + str(self.cooperationLevel()))
            logging.debug('Std Dev - Coop Lvl: ' + str(np.std(self.coopLvlShortHistory)))
            #generateChart(self.currentLatticeStrategy(), False, True, '')
            if loop % 10 == 0 or loop in (1, 5, 10, 20 , 50):
                generateChart(self.currentLatticeStrategy(), True, False, './' + self.directoryOutput + '/' + str(loop) + '-')
        logging.info('Simulation finished')
        self.saveSummary(loop)
    
    def payoffs(self):
        return [p.payoff for p in self.lattice.flat]
    
    def cooperationLevel(self):
        return self.currentLatticeStrategy().count(coop) / len(self.currentLatticeStrategy())
    
    def saveSummary(self, loop):
        target = open('./' + self.directoryOutput + '/summary.out', 'w')
        target.write(self.summary(loop))
        target.close()
    
    def summary(self, loop):
        summary = 'Loops: ' + str(loop) + '\n'
        summary += 'Avg payoff: ' + str(reduce(lambda x, y: x + y, self.payoffs()) / len(self.payoffs())) + '\n'
        summary += 'Payoffs: ' + str(self.payoffs()) + '\n'
        return summary

def generateChart(arr, save, plot, name):
    data = np.asarray([x == dfct for x in arr]).reshape(size,size)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(data)
    ax.imshow(data, cmap='Greys',  interpolation='nearest')
    if save:
        fig.savefig(name + '.png')
    if plot:
        fig.show()
    plt.cla()
    fig.clf()
    plt.clf()
    plt.close(fig)


In [2]:
avgVal = 100  # Average value used to measure the level of cooperation
size = 8  # Latice size
temp = 10  # Temptation payoff
rwrd = 7  # Reward payoff
suck = 0  # Sucker's payoff
pnsh = 0  # Punishment payoff

def runSinglePrisionerDilemma():
    prisionersDilemmaGame = Game(2, np.array([((rwrd, rwrd), (suck, temp)),((temp, suck), (pnsh, pnsh))]), {coop: 0, dfct: 1})
    sim = Simulator(size, prisionersDilemmaGame, mooreSize, moore, avgVal)
    sim.run()
    generateChart(sim.currentLatticeStrategy(), True, False, './' + sim.directoryOutput + '/final')
    # Plot cooperation level avg
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(sim.cooperationLevelData)
    fig.savefig('./' + sim.directoryOutput + '/coopLvl.png')
    plt.draw()
    ax.clear()
    plt.cla()
    fig.clf()
    plt.clf()
    plt.close(fig)


for i in range(0, 10):
    for j in (4, 8, 12, 20):
        size = j
        runSinglePrisionerDilemma()
    


temp = 10  # Temptation payoff
rwrd = 7  # Reward payoff
suck = 3  # Sucker's payoff
pnsh = 0  # Punishment payoff

snowdriftGame = Game(2, np.array([((rwrd, rwrd), (suck, temp)), ((temp, suck), (pnsh, pnsh))]), {coop: 0, dfct: 1})
sim = Simulator(size, snowdriftGame, mooreSize, moore, avgVal)

INFO:root:Creating instance of simulator
INFO:root:Instance of simulator created successfully
INFO:root:Starting to run simulator
/usr/lib/python3.4/site-packages/numpy/core/_methods.py:83: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
INFO:root:Simulation finished
INFO:root:Creating instance of simulator
INFO:root:Instance of simulator created successfully
INFO:root:Starting to run simulator
INFO:root:Simulation finished
INFO:root:Creating instance of simulator
INFO:root:Instance of simulator created successfully
INFO:root:Starting to run simulator
INFO:root:Simulation finished
INFO:root:Creating instance of simulator
INFO:root:Instance of simulator created successfully
INFO:root:Starting to run simulator


KeyboardInterrupt: 